# Загрузка Pandas и очистка данных

In [424]:
import pandas as pd
import numpy as np
import json

In [425]:
original_df = pd.read_csv('data/main_task.csv')


In [426]:
print(original_df.info())
original_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


In [427]:
# На первом этапе для создания такого датафрейма давайте просто удалим столбцы, содержащие данные типа object, и заполним пропущенные значения (None или NaN) каким-то одним значением (нулём или средним арифметическим) для  всего столбца.

object_cols = list(original_df.select_dtypes(include=['object']).columns)
df = original_df.drop(labels=object_cols, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  37457 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели

In [428]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [429]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['Rating'], axis = 1)
y = df['Rating']

In [430]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

# Создаём, обучаем и тестируем модель

In [431]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [432]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [433]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.6178861275793651


Для упрощения тестирования объеденим код в функцию принимающую датафрейм и возвращающую значение переменной MAE.

In [434]:
def calc_mae(df):
    # Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
    X = df.drop(['Rating'], axis = 1)
    y = df['Rating']
    # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
    # Для тестирования мы будем использовать 25% от исходного датасета.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
    # Создаём модель
    regr = RandomForestRegressor(n_estimators=100)
    # Обучаем модель на тестовом наборе данных
    regr.fit(X_train, y_train)
    # Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
    # Предсказанные значения записываем в переменную y_pred
    y_pred = regr.predict(X_test)
    # Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
    # Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
    return metrics.mean_absolute_error(y_test, y_pred)

Посчитаем значение MAE для замены пропусков разными статистиками:

In [435]:
filler = 0
print('0:\t', calc_mae(df.fillna(filler)))
filler = df['Number of Reviews'].mean()
print('mean = ', filler)
print('mean:\t', calc_mae(df.fillna(filler)))
filler = df['Number of Reviews'].median()
print('median = ', filler)
print('median:\t', calc_mae(df.fillna(filler)))
modes = df['Number of Reviews'].mode().values
print('modes: ', modes)
for mode in modes:
    filler = mode
    print('mode = ', filler)
    print('mode:\t', calc_mae(df.fillna(filler)))

0:	 0.42666655972222217
mean =  124.82547988359985
mean:	 0.4326581212301587
median =  33.0
median:	 0.427915155952381
modes:  [2.]
mode =  2.0
mode:	 0.43203169067460323


## С одной стороны, всё просто. Задачу, которая стоит перед вами, можно свести к трём пунктам:

Удалить из датафрейма столбцы, данные в которых представлены не числами (это вы уже сделали, и нужно просто повторить знакомые действия, но в этот раз выполнить данный шаг в последнюю очередь).
Избавиться от пропущенных (None) значений (на предыдущем шаге мы делали это самым грубым из всех возможных способов; сейчас попробуем подойти к процессу более гибко).
Создать новые столбцы с данными, используя для этого информацию, содержащуюся в других столбцах датафрейма (например, можно добавить столбец, сообщающий, сколько дней прошло со дня публикации последнего отзыва, отображённого на сайте).
С другой стороны, в этом задании масса подводных камней.


## Строковые данные
В исходном наборе данных всего три столбца содержат числовые данные, причём один из этих столбцов — это целевая переменная, значение которой должна предсказывать наша модель. Так что просто удалить все object-значения и считать задачу выполненной не получится. Для создания качественной модели нам сначала придётся очень основательно поработать со строковыми данными и извлечь из них как можно больше информации, которую можно представить в числовом виде.


## Пропущенные значения
Мы уже говорили о том, что в задачах по машинному обучению принято не удалять строки с пустыми значениями, а заполнять их максимально близкими к реальности данными. Как найти такие данные — большой вопрос, требующий не только знания синтаксиса, но и креативности, изобретательности, хорошего понимания контента, а иногда ещё и интуиции. 


## Новые признаки
Это, пожалуй, самая сложная, но и самая интригующая, увлекательная и творческая часть работы на данном шаге. Создание новых признаков (Feature Engineering) потребует от вас не только хорошего владения разными библиотеками Python, но и способность вникать в контент, умение подключать к работе интуицию, творческий подход и готовность к экспериментам.

Для удобства нормальзуем названия колонок, приведем их к snake_case.

In [436]:
df = pd.read_csv('data/main_task.csv')
df.columns = [col.lower().replace(' ', '_') for col in list(original_df.columns.values)]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   restaurant_id      40000 non-null  object 
 1   city               40000 non-null  object 
 2   cuisine_style      30717 non-null  object 
 3   ranking            40000 non-null  float64
 4   rating             40000 non-null  float64
 5   price_range        26114 non-null  object 
 6   number_of_reviews  37457 non-null  float64
 7   reviews            40000 non-null  object 
 8   url_ta             40000 non-null  object 
 9   id_ta              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB
None


,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


### Вопросы о ценах
Сколько вариантов непустых значений встречается в столбце Price Range?

In [437]:
df.price_range.value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: price_range, dtype: int64

price_range - явно ординальный признак, заменим значения на следующие:
* `$` = 1
* `$$ - $$$` = 2
* `$$$$` = 3

In [438]:
def nomalize_price_range(value):
    if value == '$':
        return 1
    elif value == '$$ - $$$':
        return 2
    elif value == '$$$$':
        return 3
    return value

df.price_range = df.price_range.apply(nomalize_price_range)
df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963


## Вопрос о городах

In [439]:
len(df.city.unique())

31

## Вопросы о кухнях
Сколько типов кухонь представлено в наборе данных?

In [440]:
def string_to_list(str):
    return str[1:-1].replace("'", "").split(', ')

cuisines = set()
df.cuisine_style.dropna().apply(lambda c: cuisines.update(string_to_list(c)))
print(len(cuisines))


125


In [441]:
df['cuisine_style_count'] = df.cuisine_style.apply(lambda c: 1 if c is np.nan else len(string_to_list(c)))
df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,cuisine_style_count
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3


Какая кухня представлена в наибольшем количестве ресторанов? Введите название кухни без кавычек или апострофов.


In [442]:
cuisines_rating = {}

def update_cuisines_rating(cuisines):
    for c in cuisines:
        cuisines_rating[c] = cuisines_rating[c] + 1

for cuisine in cuisines:
    cuisines_rating[cuisine] = 0

df.cuisine_style.dropna().apply(lambda c: update_cuisines_rating(string_to_list(c)))

most_popular = None
most_popular_rating = 0
for c in cuisines_rating:
    rating = cuisines_rating[c]
    if rating > most_popular_rating:
        most_popular_rating = rating
        most_popular = c

print(most_popular)


Vegetarian Friendly


Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [443]:
df['cuisine_style_count'].mean()

2.6224

## Вопросы об отзывах

Когда был оставлен самый свежий отзыв? Введите ответ в формате yyyy-mm-dd.

In [444]:
def get_dates_list(x):
    dates = x.split('], [')[1]
    dates = dates[:-2]
    dates = dates.replace("'", "")
    return dates.split(', ')

def get_first_review(x):
    dates = get_dates_list(x)
    return pd.to_datetime(dates[-1]) if len(dates) >= 1 else np.NaN

def get_last_review(x):
    dates = get_dates_list(x)
    return pd.to_datetime(dates[0]) if len(dates) >= 1 else np.NaN

df['first_review'] = df.reviews.apply(lambda x: get_first_review(x))
df['last_review'] = df.reviews.apply(lambda x: get_last_review(x))

df.last_review.max()

Timestamp('2018-02-26 00:00:00')

Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [445]:
def get_time_delta(x):
    return x['last_review'] - x['first_review']

df = df.assign(review_time_delta = lambda x: x['last_review'] - x['first_review'])
df['review_time_delta'] = df['review_time_delta'].apply(lambda x: x.days)
df.head()
df.loc[df.review_time_delta == df.review_time_delta.max()]


,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,cuisine_style_count,first_review,last_review,review_time_delta
7990,id_10021,Paris,"['Japanese', 'Barbecue', 'Sushi']",10023.0,4.0,1.0,21.0,"[['Incredible buffet!', 'Used to be great, but...",/Restaurant_Review-g187147-d805046-Reviews-Yam...,d805046,3,2007-12-22,2016-10-02,3207.0


In [396]:
fillers = {
    'price_range': df.price_range.median(),
    'number_of_reviews': 0,
    'review_time_delta': 0
}
df = df.fillna(value=fillers)
df.info()

clean_df = df

object_cols = list(df.select_dtypes(include=['object', 'datetime64[ns]', 'timedelta64[ns]']).columns)
df = df.drop(labels=object_cols, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   restaurant_id        40000 non-null  object        
 1   city                 40000 non-null  object        
 2   cuisine_style        30717 non-null  object        
 3   ranking              40000 non-null  float64       
 4   rating               40000 non-null  float64       
 5   price_range          26114 non-null  float64       
 6   number_of_reviews    37457 non-null  float64       
 7   reviews              40000 non-null  object        
 8   url_ta               40000 non-null  object        
 9   id_ta                40000 non-null  object        
 10  cuisine_style_count  40000 non-null  int64         
 11  first_review         33529 non-null  datetime64[ns]
 12  last_review          33529 non-null  datetime64[ns]
 13  review_time_delta    33529 non-

Какое значение метрики MAE вам удалось получить на этом этапе?

In [397]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['rating'], axis = 1)
y = df['rating']
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)
# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
metrics.mean_absolute_error(y_test, y_pred)

0.38873262162698413

## Dummy

In [398]:
clean_df.head()

,restaurant_id,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,url_ta,id_ta,cuisine_style_count,first_review,last_review,review_time_delta
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.0,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,3,2017-11-20,2017-12-31,41.0
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,1,2016-06-19,2017-07-06,382.0
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,3.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,7,2018-01-06,2018-01-08,2.0
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,1,NaT,NaT,NaN
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.0,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,3,2017-02-19,2017-11-18,272.0


Добавим dummy признаки по городам

In [399]:
df = clean_df
print(df.columns)
city_dummies = pd.get_dummies(df.city)
df = df.join(city_dummies)
print(df.columns)

Index(['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
       'price_range', 'number_of_reviews', 'reviews', 'url_ta', 'id_ta',
       'cuisine_style_count', 'first_review', 'last_review',
       'review_time_delta'],
      dtype='object')
Index(['restaurant_id', 'city', 'cuisine_style', 'ranking', 'rating',
       'price_range', 'number_of_reviews', 'reviews', 'url_ta', 'id_ta',
       'cuisine_style_count', 'first_review', 'last_review',
       'review_time_delta', 'Amsterdam', 'Athens', 'Barcelona', 'Berlin',
       'Bratislava', 'Brussels', 'Budapest', 'Copenhagen', 'Dublin',
       'Edinburgh', 'Geneva', 'Hamburg', 'Helsinki', 'Krakow', 'Lisbon',
       'Ljubljana', 'London', 'Luxembourg', 'Lyon', 'Madrid', 'Milan',
       'Munich', 'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm',
       'Vienna', 'Warsaw', 'Zurich'],
      dtype='object')


## Добавим dummy признаки по кухням
Для отдельных признаков возьмем типы кухни, которые представлены хотя бы в 5% ресторанов

In [400]:
top_cuisines = []

df.head()
for cuisine in cuisines_rating:
    if cuisines_rating[cuisine] > 40000 * .05:
        top_cuisines.append(cuisine)

for cuisine in top_cuisines:
    dummy = df['cuisine_style'].apply(lambda x: cuisine in x if not pd.isna(x) else False)
    df[cuisine] = dummy

object_cols = list(df.select_dtypes(include=['object', 'datetime64[ns]', 'timedelta64[ns]']).columns)
df = df.drop(labels=object_cols, axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 50 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ranking              40000 non-null  float64
 1   rating               40000 non-null  float64
 2   price_range          26114 non-null  float64
 3   number_of_reviews    37457 non-null  float64
 4   cuisine_style_count  40000 non-null  int64  
 5   review_time_delta    33529 non-null  float64
 6   Amsterdam            40000 non-null  uint8  
 7   Athens               40000 non-null  uint8  
 8   Barcelona            40000 non-null  uint8  
 9   Berlin               40000 non-null  uint8  
 10  Bratislava           40000 non-null  uint8  
 11  Brussels             40000 non-null  uint8  
 12  Budapest             40000 non-null  uint8  
 13  Copenhagen           40000 non-null  uint8  
 14  Dublin               40000 non-null  uint8  
 15  Edinburgh            40000 non-null 

In [401]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df.drop(['rating'], axis = 1)
y = df['rating']
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)
# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
metrics.mean_absolute_error(y_test, y_pred)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').